In [3]:
import json
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from mixpanel import Mixpanel

# Credentials
project_root = os.path.dirname(os.getcwd())
dotenv_path = os.path.join(project_root, '.env.local')
load_dotenv(dotenv_path=dotenv_path)
SA_USERNAME = os.getenv("MIXPANEL_SERVICE_ACCOUNT_USERNAME").strip()
SA_SECRET = os.getenv("MIXPANEL_SERVICE_ACCOUNT_SECRET").strip()
PROJECT_ID = os.getenv("MIXPANEL_PROJECT_ID").strip()

# Endpoint
API_ENDPOINT = 'https://data.mixpanel.com/api/2.0/export'
headers = {"accept": "text/plain"}

# Date range
params = {
    'from_date': '2025-08-01',
    'to_date': '2025-08-07',
    'project_id': PROJECT_ID
}

# Make the request
response = requests.get(
    API_ENDPOINT,
    headers=headers,
    params=params,
    auth=(SA_USERNAME, SA_SECRET)
)

if response.status_code == 200:
    raw_data = []
    lines = response.text.strip().split('\n')
    for line in lines:
        try:
            # Each line is a separate event
            raw_data.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Warning: Could not decode line: {line}")
    
    df = pd.DataFrame(raw_data)
    print(f"Successfully fetched {len(df)} events.")
    print(df.head())
else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)

Successfully fetched 23792 events.
                               event  \
0           ApplicantViewedHelpTopic   
1  ApplicantClickedCBVInvitationLink   
2                        CbvPageView   
3           ApplicantViewedAgreement   
4          ApplicantConsentedToTerms   

                                          properties  
0  {'time': 1754388692, 'distinct_id': '241211968...  
1  {'time': 1754446410, 'distinct_id': 'applicant...  
2  {'time': 1754446410, 'distinct_id': 'applicant...  
3  {'time': 1754446411, 'distinct_id': 'applicant...  
4  {'time': 1754446420, 'distinct_id': 'applicant...  
